### This jupyter notebooks provides the code for classifying ECG signals using the Discrete Wavelet Transform.
### To get some more background information, please have a look at the accompanying blog-post:
### http://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy.fftpack import fft
from IPython.display import display

import pywt
import scipy.stats

import datetime as dt
from collections import defaultdict, Counter

from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def get_uci_har_features(dataset, labels, waveletname):
    uci_har_features = []
    for signal_no in range(0, len(dataset)):
        features = []
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            list_coeff = pywt.wavedec(signal, waveletname)
            for coeff in list_coeff:
                features += get_features(coeff)
        uci_har_features.append(features)
    X = np.array(uci_har_features)
    Y = np.array(labels)
    return X, Y

def get_train_test(df, y_col, x_cols, ratio):
    """ 
    This method transforms a dataframe into a train and test set, for this you need to specify:
    1. the ratio train : test (usually 0.7)
    2. the column with the Y_values
    """
    mask = np.random.rand(len(df)) < ratio
    df_train = df[mask]
    df_test = df[~mask]
       
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    X_train = df_train[x_cols].values
    X_test = df_test[x_cols].values
    return df_train, df_test, X_train, Y_train, X_test, Y_test

# 1. Loading the ECG Dataset
Download from here: https://github.com/mathworks/physionet_ECG_data/blob/master/ECGData.zip

In [30]:
filename = 'e:/PROF NOBERT/MPHIL Research Area/OneDrive/My Work/Algorithms/AI Projects/NILM_THESIS_CODE/siml/datasets/ECGData.mat'
ecg_data = sio.loadmat(filename)
ecg_signals = ecg_data['ECGData'][0][0][0]
ecg_labels_ = ecg_data['ECGData'][0][0][1]
ecg_labels = list(map(lambda x: x[0][0], ecg_labels_))

dict_ecg_data = defaultdict(list)
for ii, label in enumerate(ecg_labels):
    dict_ecg_data[label].append(ecg_signals[ii])

array([], shape=(1, 0), dtype=[('Data', 'O'), ('Labels', 'O')])

k=ARR
CHF
NSR


In [16]:
yval

0

# 2. Calculating the features per ECG signal
## And generating the training and test sets

In [33]:
list_labels = []
list_features = []
for k, v in dict_ecg_data.items():
    yval = list(dict_ecg_data.keys()).index(k)
    for signal in v:
        features = []
        list_labels.append(yval)
        list_coeff = pywt.wavedec(signal, 'sym5')
        for coeff in list_coeff:
            features += get_features(coeff)
        list_features.append(features)
df = pd.DataFrame(list_features)
ycol = 'y'
xcols = list(range(df.shape[1]))
df.loc[:,ycol] = list_labels

df_train, df_test, X_train, Y_train, X_test, Y_test = get_train_test(df, ycol, xcols, ratio = 0.5)

In [36]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,y
0,3.178054,0,2,-22.712281,-20.099288,-8.956881,-8.086078,-19.089152,-16.264346,5.590136,...,-0.024165,-0.007707,0.008126,0.027314,0.000084,0.000070,0.061612,0.003796,0.021607,0
3,3.178054,0,7,-64.585435,-58.006263,-50.848047,-45.546900,-54.457593,-54.483483,5.919372,...,-0.025039,-0.008367,0.008916,0.033576,0.000582,0.000026,0.026137,0.000683,0.014761,0
7,3.178054,0,1,-75.183143,-68.396601,-59.105992,-12.237017,-60.516276,-56.507414,19.503488,...,-0.048890,-0.013483,0.013196,0.075339,0.000212,-0.000011,0.108138,0.011694,0.040325,0
10,3.178054,0,5,-59.569613,-57.544691,-48.106817,-44.347055,-55.312934,-53.397834,5.956135,...,-0.039081,-0.011454,0.010844,0.043313,0.000255,0.000053,0.066835,0.004467,0.026401,0
12,3.178054,0,9,-60.528072,-55.828510,-51.275827,-49.326197,-52.828786,-53.907534,3.748489,...,-0.020128,-0.008583,0.008579,0.020369,0.000302,-0.000002,0.014325,0.000205,0.010592,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,3.178054,1,1,-6.401101,-5.311682,-4.897954,14.492290,-4.996879,-2.344086,7.109631,...,-0.038079,-0.006668,0.004726,0.028440,-0.001052,-0.000896,0.031971,0.001022,0.014216,2
153,3.178054,3,1,-6.005082,-0.222702,1.467056,45.230810,0.920026,6.413290,16.500249,...,-0.059702,-0.004862,0.003670,0.027376,-0.000217,-0.000095,0.068661,0.004714,0.021743,2
154,3.178054,1,4,-5.260557,-4.876815,-2.136932,9.803121,-4.660283,-2.170425,5.202046,...,-0.054408,-0.006726,0.005232,0.044908,-0.000820,-0.000835,0.039651,0.001572,0.018272,2
156,3.178054,0,7,-5.247243,-3.249783,-2.475782,-2.021698,-2.985742,-3.107760,1.005123,...,-0.077495,-0.010374,0.010042,0.096762,-0.000245,-0.000126,0.139928,0.019580,0.047543,2


# 3. Training the Gradient Boosting Classifier

In [35]:
cls = GradientBoostingClassifier(n_estimators=10000)
cls.fit(X_train, Y_train)
train_score = cls.score(X_train, Y_train)
test_score = cls.score(X_test, Y_test)
print("The Train Score is {}".format(train_score))
print("The Test Score is {}".format(test_score))

The Train Score is 1.0
The Test Score is 0.9240506329113924
